In [1]:
from scipy import io
import numpy as np

In [2]:
X = io.loadmat('origin_X.mat')
Y = io.loadmat('origin_Y.mat')

In [3]:
Data_X = X['X']
Data_Y = Y['Y']

In [4]:
import pandas as pd

In [5]:
Data_X = pd.DataFrame(Data_X)
Data_Y = pd.DataFrame(Data_Y)

In [6]:
Label = pd.DataFrame(-np.ones(np.shape(Data_Y)))

In [7]:
Label.loc[Data_Y[0] != 5] = 1

In [8]:
import math

In [9]:
num_folds = 10
subset_size = math.trunc(len(Data_X)/num_folds)

In [10]:
subset_size

5920

In [11]:
Data_X = np.array(Data_X)
Label = np.array(Label)

In [12]:
np.shape(Data_X)

(59205, 24)

In [13]:
np.shape(Label)

(59205, 1)

In [14]:
from sklearn.model_selection import KFold

In [15]:
kf = KFold(n_splits=10)

In [16]:
from sklearn.svm import SVC

In [17]:
from sklearn.metrics import confusion_matrix

In [27]:
confu_mat = list()

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'kernel':('linear','rbf','sigmoid'),'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(class_weight = 'balanced'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [30]:
# svc_param_selection(Data_X, Label, 5)

In [31]:
for train_index, test_index in kf.split(Data_X):
    X_train, X_test = Data_X[train_index], Data_X[test_index]
    y_train, y_test = Label[train_index], Label[test_index]
    y_train = np.ravel(y_train)
    
    clf = SVC(class_weight = 'balanced',C=5)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    score  = clf.score(X_test, y_test)
    print(score)
    
    conmat = confusion_matrix(y_test, y_pred)
    confu_mat.append(conmat)
    print(conmat)

0.505488937679446
[[2653 2703]
 [ 225  340]]
0.532680290491471
[[2870 2541]
 [ 226  284]]
0.5306536058098295
[[2811 2565]
 [ 214  331]]
0.5176490457692957
[[2727 2639]
 [ 217  338]]
0.48978213139672355
[[2581 2820]
 [ 201  319]]
0.5216216216216216
[[2772 2613]
 [ 219  316]]
0.5430743243243243
[[2879 2481]
 [ 224  336]]
0.4864864864864865
[[2563 2837]
 [ 203  317]]
0.5244932432432432
[[2789 2591]
 [ 224  316]]
0.5162162162162162
[[2709 2661]
 [ 203  347]]


In [32]:
sum_confu = sum(confu_mat)

In [33]:
specificity = sum_confu[0,0] / (sum_confu[0,0]+sum_confu[0,1])
sensitivity = sum_confu[1,1] / (sum_confu[1,1]+sum_confu[1,0])

In [34]:
print('specificity: {0}, sensitivity: {1}'.format(specificity,sensitivity))

specificity: 0.5083914134374129, sensitivity: 0.6007407407407407


In [35]:
sum_confu

array([[27354, 26451],
       [ 2156,  3244]])